In [ ]:
import numpy as np
import qutip
from matplotlib import pyplot, animation


# Function to calculate the Bloch vector from a quantum state
def state_to_bloch_vector(state):
    # Pauli matrices
    sx = qutip.sigmax()
    sy = qutip.sigmay()
    sz = qutip.sigmaz()

    # Calculate the expectation values <σx>, <σy>, <σz>
    bloch_x = qutip.expect(sx, state)
    bloch_y = qutip.expect(sy, state)
    bloch_z = qutip.expect(sz, state)

    # Return the Bloch vector (x, y, z)
    return [bloch_x, bloch_y, bloch_z]


# Function to animate the Bloch sphere with a list of states
def animate_bloch_trajectory(
    states, theta=np.pi / 4
):  # Define theta here (e.g., 45 degrees)
    # Create the figure and 3D plot
    fig = pyplot.figure()
    ax = fig.add_subplot(projection="3d")
    sphere = qutip.Bloch(axes=ax)

    # Convert each state to a Bloch vector
    bloch_vectors = [state_to_bloch_vector(qutip.Qobj(state)) for state in states]

    # Define the update function for animation
    def animate(i):
        sphere.clear()
        # Add a reference vector in the direction of theta (optional)
        sphere.add_vectors([np.sin(theta), 0, np.cos(theta)])
        # Add the trajectory points up to step i
        sphere.add_points([bv[: i + 1] for bv in zip(*bloch_vectors)])
        sphere.make_sphere()
        return ax

    # Create the animation
    ani = animation.FuncAnimation(
        fig, animate, frames=len(bloch_vectors), blit=False, repeat=False
    )

    # Save the animation as an mp4 file
    ani.save("bloch_sphere_trajectory.mp4", fps=20)


# Example: List of quantum states to animate
def create_test_states():
    from qutip import basis

    # |0> state
    initial_state = basis(2, 0).full()

    # Superposition state
    superposition_state = (basis(2, 0) + basis(2, 1)).unit().full()

    # |1> state
    final_state = basis(2, 1).full()

    return [initial_state, superposition_state, final_state]


# List of quantum states
states = create_test_states()

# Animate the trajectory of these states
animate_bloch_trajectory(states)

In [ ]:
import numpy as np

# Example: |psi_initial> and |psi_target> as numpy arrays
psi_initial = np.array([0, 0, 0, 1])  # |11>
psi_target = np.array([0, 0, 1, 0])  # |10>

# Fidelity calculation
fidelity = np.abs(np.dot(np.conjugate(psi_target), psi_initial)) ** 2
print(f"Fidelity: {fidelity:.5f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Define the fidelity range from very low (close to 0) to almost 1
fidelity = np.linspace(0, 1, 1000)  # Avoid exact 1 to prevent log(0)

# Calculate the logarithmic infidelity
log_infidelity = -np.log10(1 - fidelity)

# Plot the function
plt.figure(figsize=(8, 5))
plt.plot(fidelity, log_infidelity, label=r"$-\log_{10}(1-F)$", color="blue")
plt.xlabel("Fidelity (F)")
plt.ylabel("Logarithmic Infidelity")
plt.title("Logarithmic Infidelity as a Function of Fidelity")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
basis_states = {
    "|00>": np.array([1, 0, 0, 0]),
    "|01>": np.array([0, 1, 0, 0]),
    "|10>": np.array([0, 0, 1, 0]),
    "|11>": np.array([0, 0, 0, 1]),
}


cnot_matrix = np.array(
    [
        [1, 0, 0, 0],  # |00> -> |00>
        [0, 1, 0, 0],  # |01> -> |01>
        [0, 0, 0, 1],  # |10> -> |11>
        [0, 0, 1, 0],  # |11> -> |10>
    ]
)

transitions = [("|00>", "|00>"), ("|01>", "|01>"), ("|10>", "|11>"), ("|11>", "|10>")]

# Perform explicit fidelity calculations for each specified transition
explicit_fidelities = {}
for init_label, target_label in transitions:
    # Retrieve initial and target states
    init_state = basis_states[init_label]
    target_state = basis_states[target_label]

    # Apply CNOT to the initial state
    final_state = np.dot(cnot_matrix, init_state)

    # Perform the fidelity calculation explicitly
    fidelity = np.abs(np.vdot(target_state, final_state)) ** 2  # |<target|final>|^2
    explicit_fidelities[f"{init_label} -> {target_label}"] = fidelity

explicit_fidelities

In [ ]:
import torch
states = [
    np.array([[1, 0], [0, 1j]]),  # Identity with an imaginary element
    np.array([[0, 1j], [1, 0]]),  # Swap gate with an imaginary element
]

# Process states
processed_states = torch.FloatTensor(
    np.array([np.concatenate([s.flatten().real, s.flatten().imag]) for s in states])
)

print(processed_states)

In [ ]:
import numpy as np
from scipy.linalg import logm


# Define a helper function to calculate von Neumann entropy
def von_neumann_entropy(density_matrix):
    # Ensure the density matrix is Hermitian
    eigenvalues = np.linalg.eigvalsh(density_matrix)
    # Filter small eigenvalues for numerical stability
    eigenvalues = eigenvalues[eigenvalues > 1e-10]
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    return entropy


# Define a Bell state density matrix
bell_state = np.array([[1, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 1]]) / 2

# Apply an Ry(theta) gate to the second qubit
theta = np.pi / 4  # Example value for theta
Ry = np.array(
    [[np.cos(theta / 2), -np.sin(theta / 2)], [np.sin(theta / 2), np.cos(theta / 2)]]
)

# Tensor product of identity and Ry to act on the second qubit
I = np.eye(2)
operator = np.kron(I, Ry)

# Updated density matrix after applying Ry(theta)
updated_state = operator @ bell_state @ operator.T
dims = (2, 2)

# Trace out one qubit (partial trace to get reduced density matrix)
def partial_trace(density_matrix, system_dim):
    dim_A, dim_B = system_dim
    reshaped_matrix = density_matrix.reshape([dim_A, dim_B, dim_A, dim_B])
    return np.trace(reshaped_matrix, axis1=1, axis2=3)


def compute_entropy_vs_theta():
    thetas = np.linspace(0, 2 * np.pi, 100)  # Theta values from 0 to 2π
    entropies = []

    for theta in thetas:
        # Define the Ry(theta) gate
        Ry = np.array(
            [
                [np.cos(theta / 2), -np.sin(theta / 2)],
                [np.sin(theta / 2), np.cos(theta / 2)],
            ]
        )

        # Apply the gate to the Bell state
        operator = np.kron(I, Ry)
        updated_state = operator @ bell_state @ operator.T

        # Compute the reduced density matrix
        reduced_density_matrix = partial_trace(updated_state, dims)

        # Compute the von Neumann entropy
        entropy = von_neumann_entropy(reduced_density_matrix)
        entropies.append(entropy)

    return thetas, entropies


# Compute entropy as a function of theta
thetas, entropies = compute_entropy_vs_theta()

# Plot the results
plt.figure(figsize=(8, 5))
plt.plot(thetas, entropies, label="Von Neumann Entropy")
plt.title("Von Neumann Entropy vs. θ")
plt.xlabel("θ (radians)")
plt.ylabel("Entropy")
plt.grid()
plt.legend()
plt.show()

In [ ]:
a = np.array([1, 0])
b = np.array([1, 0])

In [ ]:
print(np.kron(a,b))

In [ ]:
hadamard = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
I = np.eye(2)
operator = np.kron(I, hadamard)
updated_state = operator @ bell_state @ operator.T
updated_state

In [ ]:
cnot = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
updated_state = cnot @ bell_state @ cnot.T
updated_state

In [ ]:
# Re-define the sequence of operations: (Ry ⊗ I) CNOT (H ⊗ I) |00⟩
import pandas as pd
def compute_all_measures_corrected():
    thetas = np.linspace(0, 2 * np.pi, 100)  # Theta values from 0 to 2π
    entropies = []
    purities = []
    concurrences = []
    mutual_informations = []

    for theta in thetas:
        # Define the gates
        H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)  # Hadamard gate
        CNOT = np.array(
            [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
        )  # CNOT gate
        Ry = np.array(
            [
                [np.cos(theta / 2), -np.sin(theta / 2)],
                [np.sin(theta / 2), np.cos(theta / 2)],
            ]
        )  # Ry(theta)

        # Create the initial state |00⟩
        initial_state = np.array([1, 0, 0, 0]).reshape(4, 1)

        # Apply the gate sequence: (Ry ⊗ I) CNOT (H ⊗ I) |00⟩
        H_I = np.kron(H, I)  # H ⊗ I
        Ry_I = np.kron(Ry, I)  # Ry(theta) ⊗ I
        final_state = Ry_I @ CNOT @ H_I @ initial_state
        density_matrix = final_state @ final_state.T.conj()

        # Compute the reduced density matrices
        reduced_density_matrix_A = partial_trace(density_matrix, dims)
        reduced_density_matrix_B = partial_trace(
            density_matrix.T, dims
        )  # Symmetric in this case

        # Von Neumann entropy
        entropy = von_neumann_entropy(reduced_density_matrix_A)
        entropies.append(entropy)

        # # Purity
        # purity = calculate_purity(reduced_density_matrix_A)
        # purities.append(purity)

        # # Concurrence
        # concurrence = calculate_concurrence(density_matrix)
        # concurrences.append(concurrence)

        # # Mutual information
        # mutual_information = calculate_mutual_information(
        #     reduced_density_matrix_A, reduced_density_matrix_B, density_matrix
        # )
        # mutual_informations.append(mutual_information)

    return thetas, entropies, purities, concurrences, mutual_informations


# Perform the corrected calculations
(
    thetas_corrected,
    entropies_corrected,
    purities_corrected,
    concurrences_corrected,
    mutual_informations_corrected,
) = compute_all_measures_corrected()

# Organize corrected results into a DataFrame for display
corrected_results = pd.DataFrame(
    {
        "Theta (radians)": thetas_corrected,
        "Von Neumann Entropy": entropies_corrected,
        # "Purity": purities_corrected,
        # "Concurrence": concurrences_corrected,
        # "Mutual Information": mutual_informations_corrected,
    }
)

# tools.display_dataframe_to_user(
#     name="Corrected Quantum Measures as a Function of Theta",
#     dataframe=corrected_results,
# )

# Plot all corrected measures for visualization
plt.figure(figsize=(10, 6))
plt.plot(thetas_corrected, entropies_corrected, label="Von Neumann Entropy")
# plt.plot(thetas_corrected, purities_corrected, label="Purity")
# plt.plot(thetas_corrected, concurrences_corrected, label="Concurrence")
# plt.plot(thetas_corrected, mutual_informations_corrected, label="Mutual Information")
plt.title("Corrected Quantum Measures vs. θ")
plt.xlabel("θ (radians)")
plt.ylabel("Measure Value")
plt.legend()
plt.grid()
plt.show()

In [ ]:
theta = np.pi
RY = np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])
operator = np.kron(I, RY)
updated_state = operator @ updated_state @ operator.T
updated_state

In [ ]:
def _process_state(states, gate_fidelity=False):
        """
        Processes a batch of states for gate or state fidelity.

        Parameters:
        ----------
        states : list of tuple
            A list of states, where each state is either:
            - A tuple containing unitary matrices (for gate fidelity), or
            - A tuple containing state vectors (for state fidelity).
        is_gate_fidelity : bool
            If True, process the states as unitary matrices (gate fidelity).
            If False, process the states as state vectors (state fidelity).

        Returns:
        -------
        torch.Tensor
            A tensor representation of the processed batch of states.
        """
        processed_batch = []
        for state in states:
            processed_units = []
            for element in state:
                if gate_fidelity:
                    # Process unitary matrix
                    flattened_real = element.flatten().real
                    flattened_imag = element.flatten().imag
                    processed_units.append(
                        np.concatenate([
                            flattened_real,
                            flattened_imag
                        ])
                    )
                else:
                    # Process state vector
                    real_part = element.real
                    imag_part = element.imag
                    processed_units.append(
                        np.concatenate([
                            real_part,
                            imag_part
                        ])
                    )
            # Combine all processed units for this state
            processed_batch.append(np.concatenate(processed_units))

        # Convert batch to torch.Tensor
        return torch.FloatTensor(np.array(processed_batch))


In [ ]:
unitary1 = np.array([[1, 0], [0, -1]])
unitary2 = np.array([[0, 1], [1, 0]])
states = [(unitary1, unitary2)]  # Batch with one state
processed = _process_state(states, gate_fidelity=True)
print(processed)

In [ ]:
state1 = np.array([1 / np.sqrt(2), 1j / np.sqrt(2)])
state2 = np.array([0, 1])
states = [state1, state2]  # Batch with one state
processed = _process_state(states, gate_fidelity=False)
print(processed)

In [ ]:
torch.FloatTensor(np.array([np.concatenate([s.real, s.imag]) for s in states]))

In [ ]:
import numpy as np
np.random.rand()

In [ ]:
import numpy as np
from scipy.linalg import expm

# Example Hamiltonian (2-qubit system)
H = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

# Time step
t = 0


# Compute time evolution operator
def _time_evolution_operator(H, t):
    return expm(-1j * H * t)


U = _time_evolution_operator(H, t)
print("Time Evolution Operator U(t):")
print(U)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.linalg import expm
import matplotlib.pyplot as plt


# Define Hamiltonian parameters
class Hamiltonian:
    def __init__(self, omega_max, delta_max, J_max):
        self.omega_max = omega_max
        self.delta_max = delta_max
        self.J_max = J_max

    def compute(self, omega, delta, J, t):
        X = np.array([[0, 1], [1, 0]])
        Y = np.array([[0, -1j], [1j, 0]])
        Z = np.array([[1, 0], [0, -1]])
        I = np.eye(2)

        H1 = 0.5 * (omega * np.cos(t) * X + omega * np.sin(t) * Y + delta * Z)
        H2 = 0.5 * (omega * np.cos(t) * X + omega * np.sin(t) * Y + delta * Z)
        return np.kron(H1, I) + np.kron(I, H2) + 0.5 * J * np.kron(Z, Z)


# DRL agent
class DRLAgent(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DRLAgent, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim),
        )

    def forward(self, x):
        return self.fc(x)


# Training loop
def train_agent(agent, env, episodes, learning_rate):
    optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
    # Lists to store metrics
    all_rewards = []
    all_losses = []
    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0
        episode_losses = []

        while not done:
            # Convert state to PyTorch tensor
            # Normalize the state before feeding it to the agent
            state_tensor = torch.tensor(state, dtype=torch.float32)
            state_tensor = (state_tensor - state_tensor.mean()) / (state_tensor.std() + 1e-5)


            # Get action from the agent
            action = agent(state_tensor).detach().numpy()

            # Take a step in the environment
            next_state, reward, done = env.step(action)
            total_reward += reward  # Accumulate total reward for the episode

            # Convert reward to PyTorch tensor
            reward_tensor = torch.tensor(
                reward, dtype=torch.float32, requires_grad=True
            )

            # Define loss and optimize policy
            loss = -reward_tensor  # Negative reward as loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Store loss
            episode_losses.append(loss.item())

            # Update state
            state = next_state

        # Store metrics for the episode
        all_rewards.append(total_reward)
        all_losses.append(sum(episode_losses) / len(episode_losses))  # Average loss

    return all_rewards, all_losses


# Environment simulation
class QuantumEnv:
    def __init__(self, hamiltonian, steps, omega_max, delta_max, J_max):
        self.hamiltonian = hamiltonian
        self.steps = steps
        self.omega_max = omega_max
        self.delta_max = delta_max
        self.J_max = J_max

    def reset(self):
        self.t = 0
        self.propagator = np.eye(4, dtype=complex)
        return self._get_state()

    def _get_state(self):
        return np.hstack(
            [self.propagator.real.flatten(), self.propagator.imag.flatten()]
        )

    def step(self, action):
        omega, delta, J = action
        H = self.hamiltonian.compute(
            omega * self.omega_max, delta * self.delta_max, J * self.J_max, self.t
        )
        self.propagator = expm(-1j * H * (1 / self.steps)) @ self.propagator
        self.t += 1 / self.steps
        done = self.t >= 1
        reward = self._get_reward()
        return self._get_state(), reward, done

    def _get_reward(self):
        target_gate = expm(
            -1j
            * np.pi
            / 4
            * np.kron(np.array([[0, -1j], [1j, 0]]), np.array([[0, -1j], [1j, 0]]))
        )
        fidelity = np.abs(np.trace(target_gate.conj().T @ self.propagator) / 4) ** 2
        return 1 - fidelity


# Main function
if __name__ == "__main__":
    # Define parameters
    omega_max = 2 * np.pi
    delta_max = 2 * np.pi
    J_max = 2 * np.pi
    steps = 20

    hamiltonian = Hamiltonian(omega_max, delta_max, J_max)
    env = QuantumEnv(hamiltonian, steps, omega_max, delta_max, J_max)
    agent = DRLAgent(input_dim=32, output_dim=3)  # Match input_dim to state vector size

    # Train the agent
    episodes = 10000
    learning_rate = 1e-4
    rewards, losses = train_agent(agent, env, episodes, learning_rate)

    # Plot results
    plt.figure(figsize=(12, 6))
    
    # Plot rewards
    plt.subplot(1, 2, 1)
    plt.plot(range(episodes), rewards, label="Reward")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title("Total Reward per Episode")
    plt.legend()

    # Plot losses
    plt.subplot(1, 2, 2)
    plt.plot(range(episodes), losses, label="Loss", color="red")
    plt.xlabel("Episode")
    plt.ylabel("Loss")
    plt.title("Loss per Episode")
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.linalg import expm


# Define Hamiltonian
class Hamiltonian:
    def __init__(self):
        pass

    def compute(self, omega, delta, J, t):
        X = np.array([[0, 1], [1, 0]])
        Y = np.array([[0, -1j], [1j, 0]])
        Z = np.array([[1, 0], [0, -1]])
        I = np.eye(2)

        H1 = 0.5 * (omega * np.cos(t) * X + omega * np.sin(t) * Y + delta * Z)
        H2 = 0.5 * (omega * np.cos(t) * X + omega * np.sin(t) * Y + delta * Z)
        return np.kron(H1, I) + np.kron(I, H2) + 0.5 * J * np.kron(Z, Z)


# Define RL Agent
class RLAgent(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.1):
        super(RLAgent, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
        )

    def forward(self, x):
        return self.fc(x)


# Define Environment
class QuantumEnv:
    def __init__(self, hamiltonian, steps, omega_max, delta_max, J_max):
        self.hamiltonian = hamiltonian
        self.steps = steps
        self.omega_max = omega_max
        self.delta_max = delta_max
        self.J_max = J_max

    def reset(self):
        self.t = 0
        self.propagator = np.eye(4, dtype=complex)
        return self._get_state()

    def _get_state(self):
        return np.hstack(
            [self.propagator.real.flatten(), self.propagator.imag.flatten()]
        )

    def step(self, action):
        omega, delta, J = action
        H = self.hamiltonian.compute(
            omega * self.omega_max, delta * self.delta_max, J * self.J_max, self.t
        )
        self.propagator = expm(-1j * H * (1 / self.steps)) @ self.propagator
        self.t += 1 / self.steps
        done = self.t >= 1
        reward = self._get_reward()
        return self._get_state(), reward, done

    def _get_reward(self):
        target_gate = expm(
            -1j
            * np.pi
            / 4
            * np.kron(np.array([[0, -1j], [1j, 0]]), np.array([[0, -1j], [1j, 0]]))
        )
        fidelity = np.abs(np.trace(target_gate.conj().T @ self.propagator) / 4) ** 2
        return -np.log10(1 - fidelity)


# Train the RL Agent
def train_agent(agent, env, episodes, learning_rate):
    optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
    all_rewards = []
    for episode in range(episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            state_tensor = torch.tensor(state, dtype=torch.float32, requires_grad=False)
            action = agent(state_tensor).detach().numpy()
            action += np.random.normal(
                0, 0.1, size=action.shape
            )  # Gaussian perturbation

            next_state, reward, done = env.step(action)
            total_reward += reward  # Accumulate the total reward for the episode

            # Use the reward to calculate the loss
            loss = -torch.tensor(reward, dtype=torch.float32, requires_grad=True)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            state = next_state

        all_rewards.append(total_reward)

    return all_rewards


# Main Script
if __name__ == "__main__":
    omega_max, delta_max, J_max = 2 * np.pi, 2 * np.pi, 2 * np.pi
    steps = 20
    hamiltonian = Hamiltonian()
    env = QuantumEnv(hamiltonian, steps, omega_max, delta_max, J_max)
    agent = RLAgent(input_dim=32, output_dim=3)

    episodes = 10000
    learning_rate = 1e-5
    rewards = train_agent(agent, env, episodes, learning_rate)

    # Plot rewards
    import matplotlib.pyplot as plt

    plt.plot(range(episodes), rewards)
    plt.xlabel("Episodes")
    plt.ylabel("Total Reward")
    plt.title("RL Agent Training Rewards")
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


# Define the RL agent as a neural network
class RLAgent(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, dropout_prob=0.1):
        super(RLAgent, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, state):
        return self.net(state)


# Define the environment
class QuantumEnvironment:
    def __init__(self, target_gate, perturbation_std=0.01):
        self.target_gate = target_gate  # e.g., R_YY(pi/4)
        self.perturbation_std = perturbation_std
        self.reset()

    def reset(self):
        self.U = np.eye(4, dtype=np.complex128)  # Initial propagator
        self.time_step = 0
        return self._get_state()

    def _get_state(self):
        real_part = self.U.real.flatten()
        imag_part = self.U.imag.flatten()
        normalized_time = np.array([self.time_step / 100.0])
        return np.concatenate([real_part, imag_part, normalized_time])

    def step(self, action):
        # Apply Gaussian perturbation to simulate noise
        perturbed_action = action + np.random.normal(0, self.perturbation_std, size=action.shape)

        # Construct a unitary evolution matrix from the perturbed action
        # For simplicity, assume action contains amplitudes for Pauli operators (X, Y, Z)
        omega_x, omega_y, omega_z = perturbed_action
        unitary = np.array([
            [np.cos(omega_z) - 1j * np.sin(omega_z), 0],
            [0, np.cos(omega_z) + 1j * np.sin(omega_z)]
        ])  # Replace this with your actual dynamics model if necessary

        # Update the propagator
        self.U = self.U @ unitary  # Matrix multiplication
        self.time_step += 1

        # Reward: similarity to the target gate
        reward = -np.linalg.norm(self.U - self.target_gate)
        done = self.time_step >= 100  # End episode after 100 steps
        return self._get_state(), reward, done


# Training loop
def train_rl_agent(env, agent, optimizer, criterion, num_episodes=500):
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0

        for t in range(100):  # Max steps per episode
            state_tensor = torch.tensor(state, dtype=torch.float32)
            action = agent(state_tensor).detach().numpy()  # Forward pass
            next_state, reward, done = env.step(action)

            # Optimize the agent
            optimizer.zero_grad()
            predicted_action = agent(state_tensor)
            loss = criterion(
                predicted_action, torch.tensor(action, dtype=torch.float32)
            )
            loss.backward()
            optimizer.step()

            total_reward += reward
            state = next_state
            if done:
                break

        print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}")


# Evaluation
def evaluate_agent(env, agent):
    state = env.reset()
    total_reward = 0

    for t in range(100):  # Max steps
        state_tensor = torch.tensor(state, dtype=torch.float32)
        action = agent(state_tensor).detach().numpy()  # Forward pass
        next_state, reward, done = env.step(action)
        total_reward += reward
        state = next_state
        if done:
            break

    print(f"Evaluation Total Reward: {total_reward}")


# Main script
if __name__ == "__main__":
    input_dim = 33  # Example: real + imag parts of 4x4 propagator + normalized time
    output_dim = 3  # Example: control amplitudes, phases, interaction terms
    agent = RLAgent(input_dim, output_dim)

    # Define the target gate (e.g., R_YY(pi/4))
    target_gate = np.array(
        [[1, 0, 0, 0], [0, 0, -1j, 0], [0, -1j, 0, 0], [0, 0, 0, 1]],
        dtype=np.complex128,
    )

    env = QuantumEnvironment(target_gate=target_gate)
    optimizer = optim.Adam(agent.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Train the agent
    train_rl_agent(env, agent, optimizer, criterion, num_episodes=100)

    # Evaluate the agent in a noise-free environment
    noise_free_env = QuantumEnvironment(target_gate=target_gate, perturbation_std=0.0)
    evaluate_agent(noise_free_env, agent)

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.linalg import expm
from torch.distributions import Categorical


# Define the PPO Actor-Critic model
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim, hidden_dim=128):
        super(ActorCritic, self).__init__()
        # Common network
        self.shared = nn.Sequential(nn.Linear(input_dim, hidden_dim), nn.ReLU())
        # Actor network
        self.actor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1),
        )
        # Critic network
        self.critic = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )

    def forward(self, state):
        shared_out = self.shared(state)
        return self.actor(shared_out), self.critic(shared_out)


# Define the QuantumEnvironment
class QuantumEnvironment:
    def __init__(
        self, target_gate, T=1.0, N_max=100, Omega_max=1.0, Delta_max=1.0, J_max=1.0
    ):
        self.target_gate = target_gate
        self.T = T
        self.N_max = N_max
        self.delta_T = T / N_max
        self.Omega_max = Omega_max
        self.Delta_max = Delta_max
        self.J_max = J_max
        self.reset()

    def reset(self):
        self.U = np.eye(4, dtype=np.complex128)
        self.time_step = 0
        return self._get_state()

    def step(self, action):
        # Denormalize actions
        Omega1, Delta1, Omega2, Delta2, J = action
        Omega1 *= self.Omega_max
        Delta1 = Delta1 * 2 * self.Delta_max - self.Delta_max
        Omega2 *= self.Omega_max
        Delta2 = Delta2 * 2 * self.Delta_max - self.Delta_max
        J *= self.J_max

        # Define Hamiltonian
        H1 = Omega1 * np.kron(self._pauli_x(), np.eye(2)) + Delta1 * np.kron(
            self._pauli_z(), np.eye(2)
        )
        H2 = Omega2 * np.kron(np.eye(2), self._pauli_x()) + Delta2 * np.kron(
            np.eye(2), self._pauli_z()
        )
        H_interaction = J * np.kron(self._pauli_z(), self._pauli_z())
        H_total = H1 + H2 + H_interaction

        # Propagator update
        self.U = self.U @ expm(-1j * H_total * self.delta_T)
        self.time_step += 1

        # Reward calculation
        done = self.time_step >= self.N_max
        reward = 0
        if done:
            reward = self._calculate_reward()
        return self._get_state(), reward, done

    def _get_state(self):
        real_part = self.U.real.flatten()
        imag_part = self.U.imag.flatten()
        normalized_time = np.array([self.time_step / self.N_max])
        return np.concatenate([real_part, imag_part, normalized_time])

    def _calculate_reward(self):
        fidelity = self._calculate_fidelity()
        return -np.log10(1 - fidelity)

    def _calculate_fidelity(self):
        dim = self.U.shape[0]
        overlap = np.trace(self.target_gate.conj().T @ self.U)
        fidelity = np.abs(overlap) ** 2 / dim
        return fidelity

    @staticmethod
    def _pauli_x():
        return np.array([[0, 1], [1, 0]])

    @staticmethod
    def _pauli_y():
        return np.array([[0, -1j], [1j, 0]])

    @staticmethod
    def _pauli_z():
        return np.array([[1, 0], [0, -1]])


# PPO training function
def train_ppo(
    env, model, optimizer, num_episodes=1000, gamma=0.99, epsilon=0.2, c1=1.0, c2=0.01
):
    for episode in range(num_episodes):
        state = torch.tensor(env.reset(), dtype=torch.float32)
        log_probs, values, rewards, states, actions = [], [], [], [], []

        # Collect trajectory
        done = False
        while not done:
            policy, value = model(state)
            action_dist = Categorical(policy)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action)
            next_state, reward, done = env.step(action.numpy())

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)
            states.append(state)
            actions.append(action)
            state = torch.tensor(next_state, dtype=torch.float32)

        # Compute advantages and returns
        returns, advantages = compute_gae(rewards, values, gamma)

        # Update policy and value network
        for _ in range(4):  # PPO multiple updates
            update_ppo(
                model,
                optimizer,
                states,
                actions,
                log_probs,
                values,
                returns,
                advantages,
                epsilon,
                c1,
                c2,
            )

        print(f"Episode {episode + 1}, Total Reward: {sum(rewards):.3f}")


def compute_gae(rewards, values, gamma, tau=0.95):
    returns = []
    advantages = []
    next_value = 0
    gae = 0

    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * next_value - values[step].detach()
        gae = delta + gamma * tau * gae
        advantages.insert(0, gae)
        next_value = values[step].detach()
        returns.insert(0, gae + values[step].detach())

    returns = torch.tensor(returns, dtype=torch.float32)
    advantages = torch.tensor(advantages, dtype=torch.float32)
    return returns, advantages


def update_ppo(
    model,
    optimizer,
    states,
    actions,
    log_probs,
    values,
    returns,
    advantages,
    epsilon,
    c1,
    c2,
):
    new_log_probs, new_values = [], []
    for state, action in zip(states, actions):
        policy, value = model(state)
        action_dist = Categorical(policy)
        new_log_probs.append(action_dist.log_prob(action))
        new_values.append(value)

    new_log_probs = torch.stack(new_log_probs)
    new_values = torch.stack(new_values)

    ratios = torch.exp(new_log_probs - torch.stack(log_probs))
    surrogate1 = ratios * advantages
    surrogate2 = torch.clamp(ratios, 1 - epsilon, 1 + epsilon) * advantages
    actor_loss = -torch.min(surrogate1, surrogate2).mean()

    critic_loss = c1 * (returns - new_values).pow(2).mean()
    entropy_loss = c2 * -torch.stack([dist.entropy() for dist in new_log_probs]).mean()

    loss = actor_loss + critic_loss + entropy_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


if __name__ == "__main__":
    target_gate = expm(
        -1j
        * (
            np.kron(np.array([[0, -1j], [1j, 0]]), np.array([[0, -1j], [1j, 0]]))
            * np.pi
            / 4
        )
    )
    env = QuantumEnvironment(target_gate=target_gate)

    input_dim = 33  # 16 real, 16 imag, 1 normalized time
    action_dim = 5  # Omega1, Delta1, Omega2, Delta2, J
    model = ActorCritic(input_dim, action_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_ppo(env, model, optimizer, num_episodes=1000)

TypeError: iteration over a 0-d array

In [21]:
import numpy as np
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
final_unitary=np.eye(2)
dim = final_unitary.shape[0]  # Dimension of the unitary matrix
# Fidelity calculation
overlap = np.trace(np.dot(H.conj().T, final_unitary))
fidelity = np.abs(overlap) ** 2 / dim
fidelity.item()

0.0

In [20]:
H.conj().T, dim

(array([[ 0.70710678,  0.70710678],
        [ 0.70710678, -0.70710678]]),
 2)

In [5]:
import numpy as np


def gate_fidelity(final_unitary, target_unitary):
    """
    Calculate the fidelity between the accumulated propagator and the target unitary.

    Parameters:
        final_unitary (np.ndarray): The accumulated propagator matrix (unitary).
        target_unitary (np.ndarray): The target unitary matrix.

    Returns:
        float: The fidelity value.
    """
    dim = final_unitary.shape[0]  # Dimension of the unitary matrix

    # Fidelity calculation
    overlap = np.trace(np.dot(target_unitary.conj().T, final_unitary))
    fidelity = np.abs(overlap/dim) ** 2
    return fidelity


# Given matrices
final_unitary = np.array(
    [
        [0.58468829 + 0.63754837j, 0.13029964 + 0.48445194j],
        [-0.13029964 + 0.48445194j, 0.58468829 - 0.63754837j],
    ]
)

target_unitary = np.array(
    [[0.70710678 + 0.0j, 0.70710678 + 0.0j], [0.70710678 + 0.0j, -0.70710678 + 0.0j]]
)


def is_unitary(matrix):
    return np.allclose(np.eye(matrix.shape[0]), matrix.conj().T @ matrix)


print("Is final_unitary unitary?", is_unitary(final_unitary))
print("Is target_unitary unitary?", is_unitary(target_unitary))


# Calculate fidelity
fidelity = gate_fidelity(final_unitary, target_unitary)
print('fidelity', fidelity.item())  # Print fidelity
def debug_fidelity(final_unitary, target_unitary):
    print("Final Unitary:\n", final_unitary)
    print("Target Unitary:\n", target_unitary)
    print("Trace of Final Unitary:\n", np.trace(final_unitary))
    print("Trace of Target Unitary:\n", np.trace(target_unitary))
    print("Overlap:\n", np.trace(np.dot(target_unitary.conj().T, final_unitary)))
    print("Fidelity:\n", gate_fidelity(final_unitary, target_unitary))
debug_fidelity(final_unitary, target_unitary)

Is final_unitary unitary? True
Is target_unitary unitary? True
fidelity 0.6294423457075998
Final Unitary:
 [[ 0.58468829+0.63754837j  0.13029964+0.48445194j]
 [-0.13029964+0.48445194j  0.58468829-0.63754837j]]
Target Unitary:
 [[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]
Trace of Final Unitary:
 (1.16937658+0j)
Trace of Target Unitary:
 0j
Overlap:
 1.5867480527262037j
Fidelity:
 0.6294423457075998
